### 행정안전부 보도자료 데이터 수집
- 정적 페이지 데이터 수집: html -> bs, css-selector
- post 방식
- 여러개의 페이지 데이터 수집

In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
# 1. 웹서비스 분석: url
url = "https://www.mois.go.kr/frt/bbs/type010/commonSelectBoardList.do?bbsId=BBSMSTR_000000000008"
params = {
    "nttId": "0",
    "bbsTyCode": "BBST03",
    "bbsAttrbCode": "BBSA03",
    "authFlag": "Y",
    "pageIndex": "3",
    "cal_url": "/sym/cal/EgovNormalCalPopup.do",
    "searchCnd": "0",
}

In [8]:
# 2. request(url) > response: html
response = requests.post(url, params)
response

<Response [200]>

In [14]:
# 3. html(str) > bs_obj > bs_obj.select(css-selector) > text > DataFrame
dom = BeautifulSoup(response.text, "html.parser")

# 게시글 리스트 데이터 선택: 10개
elements = dom.select("#print_area > div.table_wrap.type_01 > form > table > tbody > tr")
len(elements)

10

In [24]:
# 각 데이터에서 필요한 정보 수집
element = elements[0]

datum = {
    "no": element.select("td")[0].text.strip(),
    "title": element.select("td")[1].text.strip(),
    "writer": element.select("td")[3].text.strip(),
    "date": element.select("td")[4].text.strip(),
    "pv": element.select("td")[5].text.strip(),
    "link": "https://www.mois.go.kr"+element.select("td")[1].select_one("a").get("href"),
}
datum

{'no': '12931',
 'title': '사이버대학 성적·졸업증명서도 모바일 전자증명서로 발급한다',
 'writer': '공공지능정책과',
 'date': '2022.02.15.',
 'pv': '812',
 'link': 'https://www.mois.go.kr/frt/bbs/type010/commonSelectBoardArticle.do;jsessionid=vRT-XYZktDGXNFQwF4CPHmnF.node30?bbsId=BBSMSTR_000000000008&nttId=90455'}

In [25]:
# [{row1}, {row2}]
data = []
for element in elements:
    data.append({
        "no": element.select("td")[0].text.strip(),
    "title": element.select("td")[1].text.strip(),
    "writer": element.select("td")[3].text.strip(),
    "date": element.select("td")[4].text.strip(),
    "pv": element.select("td")[5].text.strip(),
    "link": "https://www.mois.go.kr"+element.select("td")[1].select_one("a").get("href"),
    })

df = pd.DataFrame(data)
df.head()

,no,title,writer,date,pv,link
0,12931,사이버대학 성적·졸업증명서도 모바일 전자증명서로 발급한다,공공지능정책과,2022.02.15.,812,https://www.mois.go.kr/frt/bbs/type010/commonS...
1,12930,"2022 지방행정혁신, 주민이 체감하는 확실한 변화 만든다",지역사회혁신정책과,2022.02.15.,677,https://www.mois.go.kr/frt/bbs/type010/commonS...
2,12929,적극행정을 일상적 공직문화로 자리매김하여 국민이 현장에서 체감하는 성과를 창출하겠습니다,지역사회혁신정책과,2022.02.15.,557,https://www.mois.go.kr/frt/bbs/type010/commonS...
3,12928,'가명정보 결합전문기관 실무협의회' 출범,공공데이터정책과,2022.02.14.,647,https://www.mois.go.kr/frt/bbs/type010/commonS...
4,12927,코로나19로 지친 국민들에게 지방세 지원을 강화한다,지방세정책과,2022.02.14.,945,https://www.mois.go.kr/frt/bbs/type010/commonS...


In [32]:
# 4. function: params page
def mois(page):
    url = "https://www.mois.go.kr/frt/bbs/type010/commonSelectBoardList.do?bbsId=BBSMSTR_000000000008"
    params = {
        "nttId": "0",
        "bbsTyCode": "BBST03",
        "bbsAttrbCode": "BBSA03",
        "authFlag": "Y",
        "pageIndex": f"{page}",
        "cal_url": "/sym/cal/EgovNormalCalPopup.do",
        "searchCnd": "0",
    }
    response = requests.post(url, params)
    dom = BeautifulSoup(response.text, "html.parser")
    elements = dom.select("#print_area > div.table_wrap.type_01 > form > table > tbody > tr")
    data = []
    for element in elements:
        data.append({
            "no": element.select("td")[0].text.strip(),
        "title": element.select("td")[1].text.strip(),
        "writer": element.select("td")[3].text.strip(),
        "date": element.select("td")[4].text.strip(),
        "pv": element.select("td")[5].text.strip(),
        "link": "https://www.mois.go.kr"+element.select("td")[1].select_one("a").get("href"),
        })
    return pd.DataFrame(data)

mois(5)

,no,title,writer,date,pv,link
0,12911,2022년에도 정부혁신을 차질 없이 추진해 나가겠습니다,혁신기획과,2022.02.07.,1406,https://www.mois.go.kr/frt/bbs/type010/commonS...
1,12910,전국 최초 부동산종합정보 열람서비스 구축 등 규제해소 앞장 선 우수 지자체 선정,지방규제혁신과,2022.02.07.,1149,https://www.mois.go.kr/frt/bbs/type010/commonS...
2,12909,「2022 국가기록관리 유공」 후보를 찾습니다,기록협력과,2022.02.06.,1083,https://www.mois.go.kr/frt/bbs/type010/commonS...
3,12908,2021 정부혁신 유공 정부포상 전수식 개최,혁신기획과,2022.02.03.,1756,https://www.mois.go.kr/frt/bbs/type010/commonS...
4,12907,"""행정 칸막이 이렇게 없앴다"" 협업 우수사례집 발간",협업정책과,2022.02.03.,1704,https://www.mois.go.kr/frt/bbs/type010/commonS...
5,12906,정부대구청사 ‘무재해 결의대회’ 갖고 중대재해 근절 앞장,대구청사관리소,2022.02.03.,1396,https://www.mois.go.kr/frt/bbs/type010/commonS...
6,12905,"정부부산지방합동청사, 부산 북항에 2025년 개청",청사기획과,2022.02.02.,1405,https://www.mois.go.kr/frt/bbs/type010/commonS...
7,12904,"행안부, 대규모 예산 투입으로 자연재난 위험요소 사전 차단!",재난경감과,2022.02.02.,1760,https://www.mois.go.kr/frt/bbs/type010/commonS...
8,12903,주민 삶의 질을 높인 우수 지방자치단체 선정,자치행정과,2022.02.02.,1636,https://www.mois.go.kr/frt/bbs/type010/commonS...
9,12902,"설 연휴 중부지방 대설주의보 발표, 행안부 중대본 1단계 가동",자연재난대응과,2022.01.31.,1044,https://www.mois.go.kr/frt/bbs/type010/commonS...


In [33]:
# 5. 여러 페이지 데이터 수집
dfs = []
for page in range(1, 4):
    print(page, end="")
    dfs.append(mois(page))

123

In [37]:
result_df = pd.concat(dfs, ignore_index=True)
result_df.tail()

,no,title,writer,date,pv,link
25,12926,"행정안전부 소통협력공간, 인구감소지역 최초 선정",지역사회혁신정책과,2022.02.13.,955,https://www.mois.go.kr/frt/bbs/type010/commonS...
26,12925,국민이 제안하고 풀어가는 민생규제 혁신과제 공모전 개최,지방규제혁신과,2022.02.13.,789,https://www.mois.go.kr/frt/bbs/type010/commonS...
27,12924,이제 스마트폰으로 주민등록증 확인 가능해진다... 4월 시범실시 예정,주민과,2022.02.10.,1762,https://www.mois.go.kr/frt/bbs/type010/commonS...
28,12923,스마트워크센터 밀집현상 해소를 위해 서울역 2호점 추가 개소,스마트행정기반과,2022.02.10.,1044,https://www.mois.go.kr/frt/bbs/type010/commonS...
29,12922,"2022년 지방규제, 지역활력 제고에 초점 맞추기로",지방규제혁신과,2022.02.10.,1122,https://www.mois.go.kr/frt/bbs/type010/commonS...
